In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import hdbscan 


In [16]:
per_category = pd.read_csv('/Users/blackrose/Desktop/dsmp-2024-group-38/bs_mapped_transactional_data_new_2.csv')
per_category

,Unnamed: 0,from_totally_fake_account,monopoly_money_amount,to_randomly_generated_account,not_happened_yet_date,group
0,0,10371.0,4.00,CINEMA,01/01/2025,Cinema
1,1,88339.0,2.40,40544,01/01/2025,NaN
2,2,18555.0,2.40,85149,01/01/2025,NaN
3,3,18555.0,4.10,HIPSTER_COFFEE_SHOP,01/01/2025,Cafe
4,4,80792.0,1.95,18555,01/01/2025,NaN
...,...,...,...,...,...,...
10148275,10148275,32185.0,2.65,COFFEE_SHOP,31/12/2025,Cafe
10148276,10148276,32185.0,2.45,COFFEE_SHOP,31/12/2025,Cafe
10148277,10148277,57569.0,14.00,WINE_BAR,31/12/2025,Bar
10148278,10148278,1490.0,9.00,57569,31/12/2025,NaN


In [17]:
per_category = per_category.dropna(subset=['group'])
print(per_category)

          Unnamed: 0  from_totally_fake_account  monopoly_money_amount  \
0                  0                    10371.0                   4.00   
3                  3                    18555.0                   4.10   
5                  5                    18555.0                   4.45   
6                  6                    18555.0                   1.45   
7                  7                    18555.0                   5.00   
...              ...                        ...                    ...   
10148274    10148274                    81404.0                   1.80   
10148275    10148275                    32185.0                   2.65   
10148276    10148276                    32185.0                   2.45   
10148277    10148277                    57569.0                  14.00   
10148279    10148279                    53444.0                   2.65   

         to_randomly_generated_account not_happened_yet_date   group  
0                               CINEMA  

In [18]:
per_category = per_category.pivot_table(index='from_totally_fake_account',
                                columns='group',
                                values='monopoly_money_amount',
                                aggfunc='sum')

per_category.head(20)

group,Accessory Shop,Alcohol Store,Bar,Book Store,Butcher,Cafe,Children's Shop,Cinema,Clothing Store,Cook Shop,...,Home,Pets,Pub,Restaurant/Takeaway,Roasterie/Tea Shop,Sports Store,Streaming Service,Supermarket,Tech Store,Turkey Farm
from_totally_fake_account,,,,,,,,,,,,,,,,,,,,,
1000.0,NaN,114.98,3081.0,8.0,NaN,1955.20,160.0,252.0,145.0,NaN,...,93.0,529.0,1530.90,1613.39,68.18,NaN,107.88,681.93,NaN,NaN
1002.0,NaN,154.97,2511.5,NaN,NaN,483.75,268.0,309.0,NaN,NaN,...,NaN,NaN,1947.97,1503.67,61.82,175.0,107.88,1045.61,NaN,NaN
1006.0,NaN,107.95,2237.5,412.0,1484.98,2075.95,1480.0,227.0,65.0,149.0,...,44.0,NaN,1946.80,1576.16,31.36,NaN,107.88,931.20,NaN,NaN
1018.0,NaN,39.99,1691.5,20.0,1016.00,32.60,59.0,28.0,2009.0,NaN,...,71.0,30.0,832.00,1771.22,52.73,84.0,NaN,1288.22,NaN,NaN
1053.0,NaN,179.97,2402.0,NaN,18.00,725.85,133.0,222.0,2561.0,NaN,...,467.0,NaN,1473.21,1731.32,127.27,115.0,107.88,1119.22,79.0,NaN
1059.0,NaN,10.00,2046.0,480.0,NaN,440.25,1726.0,4.0,128.0,NaN,...,NaN,NaN,2205.98,1506.28,204.09,206.0,NaN,1085.49,NaN,NaN
1071.0,NaN,184.95,1747.0,35.0,30.00,499.65,13.0,223.0,NaN,23.0,...,NaN,NaN,1550.79,1374.50,66.36,889.0,107.88,1200.44,471.0,NaN
1078.0,NaN,33.99,2136.5,NaN,207.00,592.25,1390.0,274.0,NaN,120.0,...,NaN,NaN,1290.44,1402.88,39.09,161.0,107.88,5526.39,NaN,NaN
1086.0,NaN,283.95,1963.0,25.0,1222.00,506.40,1145.0,256.0,1749.0,NaN,...,621.0,60.0,1841.72,1741.44,114.10,117.0,107.88,1125.94,NaN,NaN


Replace NaNs where people haven't spent money to 0.

In [19]:
per_category = per_category.fillna(0)
per_category.head(10)

group,Accessory Shop,Alcohol Store,Bar,Book Store,Butcher,Cafe,Children's Shop,Cinema,Clothing Store,Cook Shop,...,Home,Pets,Pub,Restaurant/Takeaway,Roasterie/Tea Shop,Sports Store,Streaming Service,Supermarket,Tech Store,Turkey Farm
from_totally_fake_account,,,,,,,,,,,,,,,,,,,,,
1000.0,0.0,114.98,3081.0,8.0,0.00,1955.20,160.0,252.0,145.0,0.0,...,93.0,529.0,1530.90,1613.39,68.18,0.0,107.88,681.93,0.0,0.0
1002.0,0.0,154.97,2511.5,0.0,0.00,483.75,268.0,309.0,0.0,0.0,...,0.0,0.0,1947.97,1503.67,61.82,175.0,107.88,1045.61,0.0,0.0
1006.0,0.0,107.95,2237.5,412.0,1484.98,2075.95,1480.0,227.0,65.0,149.0,...,44.0,0.0,1946.80,1576.16,31.36,0.0,107.88,931.20,0.0,0.0
1018.0,0.0,39.99,1691.5,20.0,1016.00,32.60,59.0,28.0,2009.0,0.0,...,71.0,30.0,832.00,1771.22,52.73,84.0,0.00,1288.22,0.0,0.0
1053.0,0.0,179.97,2402.0,0.0,18.00,725.85,133.0,222.0,2561.0,0.0,...,467.0,0.0,1473.21,1731.32,127.27,115.0,107.88,1119.22,79.0,0.0
1059.0,0.0,10.00,2046.0,480.0,0.00,440.25,1726.0,4.0,128.0,0.0,...,0.0,0.0,2205.98,1506.28,204.09,206.0,0.00,1085.49,0.0,0.0
1071.0,0.0,184.95,1747.0,35.0,30.00,499.65,13.0,223.0,0.0,23.0,...,0.0,0.0,1550.79,1374.50,66.36,889.0,107.88,1200.44,471.0,0.0
1078.0,0.0,33.99,2136.5,0.0,207.00,592.25,1390.0,274.0,0.0,120.0,...,0.0,0.0,1290.44,1402.88,39.09,161.0,107.88,5526.39,0.0,0.0
1086.0,0.0,283.95,1963.0,25.0,1222.00,506.40,1145.0,256.0,1749.0,0.0,...,621.0,60.0,1841.72,1741.44,114.10,117.0,107.88,1125.94,0.0,0.0
